# Llama 3.1: Using the Multilingual capabilities for real-world use cases

This notebook should work well with the Data Science 3.0 kernel in SageMaker Studio.

This guide is designed to assist you in leveraging Llama 3.1's multilingual / translation capabilities with Bedrock Converse API for practical use cases for scenarios where English may not be needed. Our goal is to demonstrate how Llama 3.1 can be used for native lanugages to cover a broader expansion of regions globally. This notebook will provide a strong foundation for creating multilingual / translation applications.

We will begin by reviewing how to leverage the Amazon Bedrock Converse API. Once we are familiarized with the Converse API, we will generate some sample prompts to be used to explore Llama 3.1's new capability to address multilingual scenarios.

## Llama 3.1

Llama 3.1 is a high-quality open-source language model developed by Meta. It is designed for a wide range of natural language processing tasks, including text generation, question answering, and more.
Llama 3.1 has the following key features:
    
- Proficient in several languages, including English, French, Italian, German, Spanish, and others.
- Context window of up to 128k tokens, allowing for long-form text processing.
- Capable of handling various tasks, such as text generation, summarization, and question answering.
- Generates coherent and fluent responses.

## Multilingual Capabilities with Llama 3.1

Llama 3.1 offers robust multilingual capabilities, supporting a range of languages that enhance its utility for diverse applications. Specifically, Llama 3.1 has been trained to handle conversations in multiple languages including Spanish, Portuguese, Italian, German, Thai, French, and Hindi. This multilingual support makes it suitable for a wide array of uses across different regions. The model's multilingual abilities are part of its broader capabilities, which include state-of-the-art performance in translation tasks.

In this notebook, we will go over a few practical real-world use cases (add one more sentences)

## Prerequisites:
- AWS SDK for Python (Boto3): Ensure you have the Boto3 library installed.
- Amazon Bedrock Model ID: Obtain the model ID for the conversational AI model you wish to use.
- AWS Credentials: Configure your AWS credentials to authenticate API requests.

## Import necessary libraries

In the next cell, we will set up our environment by importing the boto3 library and necessary libraries that we will need to confiure our bedrock client to invoke our llama 3.1 model within the Amazon Bedrock Converse API.

In [1]:
import boto3
from boto3 import client
from botocore.config import Config
import json
import re
import logging
from botocore.exceptions import ClientError
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [2]:
config = Config(read_timeout=2000)
bedrock_client = boto3.client(service_name='bedrock-runtime', region_name="us-west-2", config=config)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


## Instantiate llama 3.1 model 

In [3]:
meta_llama_3_1_8b = 'meta.llama3-1-8b-instruct-v1:0'
meta_llama_3_1_70b = 'meta.llama3-1-70b-instruct-v1:0'

model_id = meta_llama_3_1_8b

## Prompting with Amazon Bedrock Converse API

The Amazon Bedrock Converse API can be used to invoke large language models on Amazon Bedrock by extracting away the special stop tokens and facilitate the creation of conversational applications by enabling the exchange of messages between users and Amazon Bedrock models.

**Key features include:**
- **Consistent Interface:**
The Converse API provides a uniform interface that works across all Amazon Bedrock models supporting messages. This consistency allows developers to write code once and use it with different models without needing to adjust for model-specific differences.

- **Turn-Based Conversations:**
The API supports multi-turn conversations, where a series of messages are exchanged between the user (acting as the user role) and the model (acting as the assistant role). This enables the development of chatbots and other conversational agents that can maintain context over multiple interactions.

- **Tool Use (Function Calling):**
The Converse API supports tool use, allowing models to request the invocation of external functions or tools. This is particularly useful for tasks that require interaction with external APIs or services. The model generates a JSON structure with the necessary parameters, which the calling application then uses to invoke the specified tool

## Prompting with the Bedrock Converse API

After setting your model ID as llama 3.1 8b model, to start interfacing with the converse API - you can put the prompt input inside the content section.

Listed below is an example of how you can change / assign the different roles and add input to the role.

#### Prompting with Converse API on Bedrock

```
{
    "role": "user | assistant",
    "content": [
        {
            "text": "string"
        }
    ]
}
```

In the next cell, we will use llama 3.1 to generate a few questions that we will use to be allow us to have a conversation in French.

In [4]:
# Define the message with content as a list
message = {
    "role": "user",
    "content": [
        {"text": "Can you provide me few example prompts of questions I would need to ask if I were to go on a trip to Paris."}
    ]
}


In [5]:
response = bedrock_client.converse(
    modelId=model_id,
    messages=[message],  # Wrap the message in a list
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 0,
        "topP": .5
    }
)

In [6]:
# Extract the 'text' content from the response
text_content = response['output']['message']['content'][0]['text']

# Replace newline escape sequences ('\\n') with actual newline characters
formatted_text = text_content.replace('\\n', '\n')

# Print the formatted text
print(formatted_text)



Here are some example prompts of questions you might want to ask if you were planning a trip to Paris:

**Accommodation**

* What are the best neighborhoods to stay in Paris for a first-time visitor?
* What are the prices like for hotels, hostels, and Airbnb apartments in Paris?
* Can you recommend any budget-friendly options for accommodation in Paris?
* How do I book a hotel or apartment in Paris, and what's the best way to ensure it's legitimate?

**Transportation**

* What are the best ways to get to Paris from the airport (CDG or ORY)?
* How does the Paris public transportation system work, and what are the best options for getting around the city?
* Can I buy a Paris Visite pass, and what does it include?
* Are there any bike-sharing programs or car rental options available in Paris?

**Food and Drink**

* What are some must-try foods and drinks in Paris, and where can I find them?
* Can you recommend any good restaurants or cafes in Paris for a budget-conscious traveler?
* How

In the cell above, we can see that Llama 3.1 was able to successfully generate some sample questions in English. We will use these questions later on in the notebook to prompt Llama 3.1 to have a full conversation in French. But first, let's go into a few of the multilingual translation basics.

## Multi-lingual use case capabilities: Translation

Llama 3.1 supports a variety of languages, allowing developers to build applications and services that cater to users from diverse linguistic backgrounds by offering multilingual capabilities that can be leveraged to create localized user interfaces and provide language-specific content. Specifically, Llama 3.1 supports languages such as English, French, German, Spanish, Italian, Portuguese, Hindi and Thai.

For translating telephone conversations into different languages like Hindi, Arabic, Korean, and Japanese, Llama 3.1 can be utilized effectively. In the next cell, we will demonstrate how a bank teller and customer conversation that is held in English can be translated to various languages.

In [7]:
conversation = """
Customer: "Hi, I recently opened a new account with your bank and I love the online banking features! Can you guide me on how to set up different alerts for each type of transaction I make?"
Teller: "Hello! I'm glad to hear you're enjoying our services. Setting up transaction alerts is simple. You can customize them through our online banking portal. Let me walk you through the process."
Customer: "I'm trying to use the mobile banking app, but it doesn't seem to be registering my login attempts. Could you help me figure out what might be going wrong?"
Teller: "Of course! Let's troubleshoot this together. First, ensure that your username and password are correct. If you're still having issues, try resetting your password or updating the app to the latest version." 
Customer: "I'm considering opening a savings account and I'm curious about the interest rates. What options are available and what are their main differences?"
Teller: "Great question! We offer several savings account options with varying interest rates. Our standard savings account has a competitive rate, while our high-yield account offers a higher rate for larger balances. I can provide more details on each."
Customer: "I wanted to report a small issue where my debit card was a bit sticky. However, your quick-start guide was super helpful and I fixed it easily by following the cleaning tips. Just thought you might want to know!"
Teller: "Thanks for letting us know. I'm glad the guide was helpful. If you encounter any more issues, feel free to reach out, and we'll be happy to assist further." 
Customer: "My new debit card stopped working within a week of receiving it. None of the transactions go through, and I need a solution or a replacement as soon as possible."
Teller: "I'm sorry to hear that. Let's resolve this quickly. Please check if the card is activated and try using a different ATM. If it still doesn't work, we'll issue a replacement card right away." 
Customer: "I've noticed that the numbers on my debit card are starting to fade after several months of use. Is this covered by the warranty?"
Teller: "Yes, card wear and tear is covered. Please bring your card to our branch, and we'll assist you with a replacement at no charge." 
Customer: "I had an issue where my account settings would reset every time I logged in. I figured out it was due to a browser conflict and resolved it by clearing my cache. Just wanted to ask if there are any new updates coming soon?"
Teller: "Good job on resolving the issue. We do roll out updates regularly to improve our online services. Keep an eye on our website or subscribe to our newsletter for the latest updates." 
Customer: "I've been having trouble with the online banking portal not saving my preferences, and it's starting to get frustrating. What can be done to ensure my settings are saved permanently?"
Teller: "I understand your frustration. Make sure you're using the latest version of your browser. Also, try adjusting your browser settings to allow cookies, which can help save your preferences."
"""

Here we will allow llama 3.1 to translate each of the following transcripted calls into different langauges by creating different prompts for Hindi, Spanish, german and italian. These prompts will all be used to ingest the conversation and translate into the native given language.

In [8]:
#prompts for language tasks/ translation

hindi_prompt = f"""
Translate the following conversation into Hindi. Your responses must be numbered, only in Hindi, and must adhere to only translating the conversation.

{conversation}
"""

spanish_prompt = f"""
Translate the following conversation into Spanish. Your responses must be numbered, only in Spanish, and must adhere to only translating the conversation.

{conversation}
"""

german_prompt = f"""
Translate the following conversation into German. Your responses must be numbered, only in German, and must adhere to only translating the conversation.

{conversation}
"""

italian_prompt = f"""
Translate the following conversation into Italian. Your responses must be numbered, only in Italian, and must adhere to only translating the conversation.

{conversation}
"""

In [9]:
def converse(prompt, inference_config):
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    response = bedrock_client.converse(
        messages=messages,
        modelId=model_id,
        inferenceConfig=inference_config
    )
    generated_text = response['output']['message']['content'][0]['text']
    print(generated_text)
    return generated_text, response

inference_config = {"temperature": 0.0, "maxTokens": 2048, "topP": 0.1}

#hindi prompt with llama 3.1
generated_text, response = converse(hindi_prompt, inference_config)



1. ग्राहक: "नमस्ते, मैंने हाल ही में आपके बैंक में एक नया खाता खोला है और मैं ऑनलाइन बैंकिंग फीचर्स से बहुत प्यार करता हूँ! कृपया मुझे बताएं कि कैसे मैं प्रत्येक प्रकार के लेनदेन के लिए अलग-अलग अलर्ट सेट कर सकता हूँ?"
2. टेलर: "नमस्ते! मैं आपको यह सुनकर खुश हूँ कि आप हमारी सेवाओं से आनंद ले रहे हैं। लेनदेन अलर्ट सेट करना सरल है। आप उन्हें हमारे ऑनलाइन बैंकिंग पोर्टल के माध्यम से कस्टमाइज़ कर सकते हैं। मैं आपको प्रक्रिया के माध्यम से चलने में मदद करूंगा।"
3. ग्राहक: "मैं मोबाइल बैंकिंग ऐप का उपयोग करने की कोशिश कर रहा हूँ, लेकिन यह मेरे लॉगिन प्रयासों को रजिस्टर नहीं कर रहा है। कृपया मुझे यह स्पष्ट करने में मदद करें कि क्या गलत हो सकता है?"
4. टेलर: "बिल्कुल! आइए इसे साथ में हल करें। सबसे पहले, सुनिश्चित करें कि आपका यूजरनेम और पासवर्ड सही हैं। यदि आप अभी भी समस्याओं का सामना कर रहे हैं, तो अपना पासवर्ड रीसेट करें या ऐप को नवीनतम संस्करण में अपडेट करें।"
5. ग्राहक: "मैं एक बचत खाता खोलने की सोच रहा हूँ और मैं जानना चाहता हूँ कि क्या विकल्प उपलब्ध हैं और उनके मुख्य अंतर क्या हैं।"
6. ट

In [10]:
#arabic prompt with llama 3.1
generated_text, response = converse(spanish_prompt, inference_config)



1. Cliente: "Hola, recientemente abrí una nueva cuenta en su banco y me encantan las características de banca en línea! ¿Puede guiarme sobre cómo configurar diferentes alertas para cada tipo de transacción que realice?"
2. Teller: "¡Hola! Me alegra saber que están disfrutando de nuestros servicios. Configurar alertas de transacciones es sencillo. Pueden personalizarlas a través de nuestra plataforma de banca en línea. Déjeme guiarlo a través del proceso."
3. Cliente: "Estoy tratando de usar la aplicación de banca móvil, pero no parece registrar mis intentos de inicio de sesión. ¿Puede ayudarme a averiguar qué podría estar pasando mal?"
4. Teller: "Por supuesto! Vamos a resolver esto juntos. Primero, asegúrese de que su nombre de usuario y contraseña sean correctos. Si aún tiene problemas, intente restablecer su contraseña o actualizar la aplicación a la última versión."
5. Cliente: "Estoy considerando abrir una cuenta de ahorros y estoy curioso sobre las tasas de interés. ¿Qué opcion

In [11]:
#german prompt with llama 3.1
generated_text, response = converse(german_prompt, inference_config)



1. Kunde: "Hallo, ich habe kürzlich ein neues Konto bei Ihrer Bank eröffnet und liebe die Funktionen im Online-Banking! Könnten Sie mir bitte zeigen, wie ich verschiedene Benachrichtigungen für jede Art von Transaktion einrichten kann?"
2. Teller: "Hallo! Ich bin froh, dass Sie unsere Dienstleistungen genießen. Die Einrichtung von Transaktionsbenachrichtigungen ist einfach. Sie können sie durch unser Online-Banking-Portal anpassen. Lassen Sie mich Ihnen durch den Prozess führen."
3. Kunde: "Ich versuche, die mobile Banking-App zu verwenden, aber sie registriert meine Anmeldeversuche nicht. Könnten Sie mir helfen, herauszufinden, was möglicherweise schief läuft?"
4. Teller: "Natürlich! Lassen Sie uns gemeinsam dieses Problem lösen. Zuerst stellen Sie sicher, dass Ihr Benutzername und Ihr Passwort korrekt sind. Wenn Sie immer noch Probleme haben, versuchen Sie, Ihr Passwort zu ändern oder die App auf die neueste Version zu aktualisieren."
5. Kunde: "Ich überlege, ein Sparkonto zu eröff

In [12]:
# italian prompt with llama 3.1
generated_text_italian, response = converse(italian_prompt, inference_config)



1. Cliente: "Ciao, ho appena aperto un nuovo conto con la vostra banca e mi piacciono molto le funzionalità di banca online! Potreste guidarmi su come impostare diversi avvisi per ogni tipo di transazione che effettuo?"
2. Teller: "Ciao! Sono felice di sentire che stai godendo dei nostri servizi. Impostare gli avvisi di transazione è semplice. Puoi personalizzarli attraverso il nostro portale di banca online. Vi lascio guidare attraverso il processo."
3. Cliente: "Sto cercando di utilizzare l'app di banca mobile, ma non sembra registrare i miei tentativi di accesso. Potreste aiutarmi a capire cosa potrebbe andare storto?"
4. Teller: "Certo! Lasciamo risolvere questo insieme. Prima assicurati che il tuo username e password siano corretti. Se hai ancora problemi, prova a resettare la password o aggiornare l'app alla versione più recente."
5. Cliente: "Sto considerando di aprire un conto di risparmio e sono curioso sulle tariffe di interesse. Quali opzioni sono disponibili e quali sono 

## Translation evaluation with Larger 70b variant

In [13]:
eval_model = meta_llama_3_1_70b

In [14]:
# Evaluate the generated text using the 70B model
eval_prompt = f"""The {conversation} text was translated from English to italian 
by converting the conversation into {generated_text_italian} text, is this an accurate translation? 
Highlight any useful translations in English that could make the new language more accurate."""

In [15]:
def converse(prompt, inference_config):
    messages = [{"role": "user", "content": [{"text": prompt}]}]
    response = bedrock_client.converse(
        messages=messages,
        modelId=eval_model,
        inferenceConfig=inference_config
    )
    generated_text = response['output']['message']['content'][0]['text']
    print(generated_text)
    return generated_text, response

inference_config = {"temperature": 0.0, "maxTokens": 2048, "topP": 0.1}


generated_text, response = converse(eval_prompt, inference_config)



The translation is generally accurate, but there are some minor errors and improvements that can be made to make it more accurate and natural-sounding in Italian. Here are some highlights:

1. "Ciao, ho appena aperto un nuovo conto con la vostra banca e mi piacciono molto le funzionalità di banca online!" - This is a good translation, but a more natural way to say it in Italian would be "Ciao, ho appena aperto un nuovo conto con la vostra banca e sono molto soddisfatto delle funzionalità di banca online!"
2. "Impostare gli avvisi di transazione è semplice. Puoi personalizzarli attraverso il nostro portale di banca online." - This is a good translation, but a more natural way to say it in Italian would be "Impostare gli avvisi di transazione è facile. Puoi personalizzarli attraverso il nostro portale di banca online."
3. "Lasciamo risolvere questo insieme. Prima assicurati che il tuo username e password siano corretti." - This is a good translation, but a more natural way to say it in

## Multilingual Chatbot functionality

LLama 3.1's new multilingual capabilities not only allow developers to translate content from one language to another, but also enables for multilingual chatbots. This is useful for scenarios where developers want to expand globally and create a chatbot to be used to speak in entirely different native languages other than english.

If you remember earlier in the notebook, we generated some sample questions that a tourist may have when they look to travel Paris. In the next cell, we will store the questions (in english) as a prompt, have Llama 3.1 translate the questions in French, and answer the questions in French.

In [16]:
questions = f"""

{formatted_text}
"""

In [17]:
# Define the message to translate and answer the questions in French
translated_convo = {
    "role": "user",
    "content": [
        {"text": f"Please translate the following questions into French and provide the answers in French:\n\n{questions}"}
    ]
}

In [18]:
# Send the request to the model
response = bedrock_client.converse(
    modelId=model_id,
    messages=[translated_convo],  # Wrap the message in a list
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 0,
        "topP": 0.5
    }
)

In [19]:
# Extract the 'text' content from the response
text_content = response['output']['message']['content'][0]['text']

# Replace newline escape sequences ('\\n') with actual newline characters
formatted_text_2 = text_content.replace('\\n', '\n')

# Print the formatted text
print(formatted_text_2)



Here are the translations of the questions into French, along with the answers:

**Hébergement**

* Quels sont les meilleurs quartiers pour séjourner à Paris pour un premier visiteur ?
* Quels sont les prix pour les hôtels, les auberges de jeunesse et les appartements Airbnb à Paris ?
* Puis-je vous recommander des options abordables pour l'hébergement à Paris ?
* Comment réserver un hôtel ou un appartement à Paris, et comment m'assurer qu'il est légitime ?

**Transport**

* Quels sont les meilleurs moyens de se rendre à Paris de l'aéroport (CDG ou ORY) ?
* Comment fonctionne le système de transport public à Paris, et quels sont les meilleurs options pour se déplacer dans la ville ?
* Puis-je acheter un Paris Visite pass, et qu'est-ce qu'il comprend ?
* Y a-t-il des programmes de location de vélos ou de location de voitures disponibles à Paris ?

**Nourriture et Boissons**

* Quels sont les plats et les boissons à essayer à Paris, et où les trouver ?
* Puis-je vous recommander des re

## Conclusion

In conclusion, Llama 3.1 represents a significant advancement in multilingual capabilities, enabling the creation of sophisticated multilingual chatbots and translation services. The model supports a wide array of languages, including English, French, German, Hindi, Italian, Portuguese, Spanish, and Thai, making it a versatile tool for global applications. Its ability to handle complex tasks with a context length of up to 128K tokens allows it to manage long conversations and provide accurate translations, which are crucial for developing multilingual chatbots.

Overall, Llama 3.1's multilingual capabilities not only facilitate effective translation across different languages but also enable the development of intelligent conversational agents that can interact seamlessly with users worldwide. This makes it an invaluable resource for businesses and developers looking to expand their reach in diverse linguistic markets.

---
## Distributors
- Amazon Web Services
- Meta